Here we use Bouguet's method.

In [ ]:
import numpy as np
import cv2

In [ ]:
lefts=[]
leftsGray=[]
rights=[]
rightsGray=[]

for i in np.arange(0,50):
    l=cv2.imread("left_"+str(i)+".png")
    r=cv2.imread("right_"+str(i)+".png")
    lefts.append(l)
    rights.append(r)
    leftsGray.append(cv2.cvtColor(l,cv2.COLOR_BGR2GRAY))
    rightsGray.append(cv2.cvtColor(r,cv2.COLOR_BGR2GRAY))

In [ ]:
def getAllChessboardCorners(images):
    res=[]
    for image in images:
        found,corners=cv2.findChessboardCorners(image,(9,6))
        if not found:
            print("Corners not found!")
        cv2.cornerSubPix(image, corners, (11,11), (-1,-1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))
        res.append(corners)
    return res

In [ ]:
pattern=np.zeros((9*6,3),np.float32)
pattern[:,:2]=np.mgrid[0:9,0:6].T.reshape(-1,2)
h,w=leftsGray[0].shape

In [ ]:
leftCorners=getAllChessboardCorners(leftsGray)
rightCorners=getAllChessboardCorners(rightsGray)
patterns=len(leftCorners)*[pattern]

In [ ]:
leftCalibWorked,leftMtx,leftDist,_,_=cv2.calibrateCamera(patterns,leftCorners,(w,h),None,None)
rightCalibWorked,rightMtx,rightDist,_,_=cv2.calibrateCamera(patterns,rightCorners,(w,h),None,None)

In [ ]:
optimalLeftMatrix,optimalLeftROI=cv2.getOptimalNewCameraMatrix(leftMtx,leftDist,(w,h),0,newImgSize=(w,h),centerPrincipalPoint=True)
optimalRightMatrix,optimalRightROI=cv2.getOptimalNewCameraMatrix(rightMtx,rightDist,(w,h),0,newImgSize=(w,h),centerPrincipalPoint=True)

In [ ]:
if not leftCalibWorked or not rightCalibWorked:
    print("Calibration didn't work")
    print("\t right:"+str(leftCalibWorked))
    print("\t left:"+str(rightCalibWorked))
else:
    print("Optimal left: \n"+str(optimalLeftMatrix))
    print("leftmatrix: \n"+str(leftMtx))
    print("Optimal right: \n"+str(optimalRightMatrix))
    print("rightmatrix: \n"+str(rightMtx))

In [ ]:
_1,_2,_3,_4,_5,R,T,E,F=cv2.stereoCalibrate(patterns,leftCorners,rightCorners,leftMtx,leftDist,rightMtx,rightDist,(w,h),flags=cv2.CALIB_FIX_INTRINSIC)

In [ ]:
print("_1: "+str(_1))
print("_2: "+str(_2))
print("_3: "+str(_3))
print("_4: "+str(_4))
print("_5: "+str(_5))

In [ ]:
R1,R2,P1,P2,Q,ROI1,ROI2=cv2.stereoRectify(leftMtx,leftDist,rightMtx,rightDist,(w,h),R,T)

In [ ]:
print("R1:"+str(R1))
print("R2:"+str(R2))
print("P1:"+str(P1))
print("P2:"+str(P2))
print("Q:"+str(Q))

In [ ]:
leftMx,leftMy=cv2.initUndistortRectifyMap(leftMtx,leftDist,R1,P1,(w,h),cv2.CV_32F)
rightMx,rightMy=cv2.initUndistortRectifyMap(rightMtx,rightDist,R2,P2,(w,h),cv2.CV_32F)

In [ ]:
resLeft=cv2.remap(leftsGray[11],leftMx,leftMy,cv2.INTER_LINEAR)
resRight=cv2.remap(rightsGray[11],rightMx,rightMy,cv2.INTER_LINEAR)

In [ ]:
cv2.imshow("left",leftsGray[11])
cv2.imshow("right",rightsGray[11])

cv2.imshow("left remap",resLeft)
cv2.imshow("right remap",resRight)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
b_matcher=cv2.StereoBM_create(numDisparities=128,blockSize=15)
r=cv2.normalize(b_matcher.compute(resLeft,resRight).astype(np.float32),None,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U)

In [ ]:
cv2.imshow("disparity BM",r)
cv2.waitKey(0)
cv2.destroyAllWindows()

See how BM parameters affect the resulting disparity map

In [ ]:
from IPython.html.widgets import interact, interactive
from IPython.display import clear_output, display

def compute_disparity(nDisparities,blksize,imNum):
    b_matcher=cv2.StereoBM_create(numDisparities=nDisparities,blockSize=blksize)
    resLeft=cv2.remap(leftsGray[imNum],leftMx,leftMy,cv2.INTER_LINEAR)
    resRight=cv2.remap(rightsGray[imNum],rightMx,rightMy,cv2.INTER_LINEAR)
    cv2.imshow("Diparity map",cv2.normalize(b_matcher.compute(resLeft,resRight).astype(np.float32),None,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U))
    cv2.waitKey(10)
    
w=interactive(compute_disparity,nDisparities=(32,64,32),blksize=(5,91,2),imNum=(0,len(leftsGray)))
clear_output(wait=True)
display(w)

Contrarily to Hartley's method, we are able to reproject the disparity map back in the world coordinates.

In [ ]:
cv2.destroyAllWindows()
im3d=cv2.reprojectImageTo3D(r,Q)

And finally, you should use numpy.savetxt to memorize your mappings!

In [ ]:
np.savetxt('leftMx',leftMx)
np.savetxt('leftMy',leftMy)

np.savetxt('rightMx',rightMx)
np.savetxt('rightMy',rightMy)